# P419 11 NumPy. Массивы II

Автор: Шабанов Павел Александрович

Email: pa.shabanov@gmail.com

URL:

+ [PythonWorld о NumPy](https://pythonworld.ru/numpy)

+ [ХАБР: NumPy, пособие для новичков](https://habr.com/post/121031/)

**Дата последнего обновления: 15.11.2018**

<a id='up'></a>
### План

1. **[Изменение элементов массивов](#changes)**
    + [присваивание значений срезам](#eq);
    + [копии массивов](#copy).
    
2. **[Объединение массивов](#merging)**
    + [Объединение векторов](#vectors)
    
3. **[Разделение массивов](#spliting)**

4. **[Универсальные функции модуля numpy](#numpy_functions)**
    + [поиск индексов numpy.where](#where);
    + [векторные срезы по нескольким осям на примере numpy.where](#vec_slice);
    + [логические условия и fancy-slicing](#logic_fancy);
    + [пример с геоданными](#geo).

5. **[Массивы с логической маской](#masked)**
    + [пространство numpy.ma](#ma);
    + [numpy.NaN](#nan);
    + [заполнение замаскированных значений](#filled).
  
6. **[Управляемый вывод представления чисел](#print_numpy)**

### Цель: 

+ дополнить базовые знания о массивах numpy массивы универсальными numpy функциями

<a id='changes'></a>
## Изменения элементов массивов
[Вверх](#up)

Базовым способом изменения значений элементов массива является присваивание через индексы - набор целочисленных чисел в квадратных скобках от имени массива.

In [ ]:
import numpy as np

one = np.arange(1, 13)
two = np.random.random((10, 20))
four = np.ones((5, 4, 3, 7))

temp = 36.6
one[2] = temp
two[2, 2] = temp
four[0, 0, 2, 2] = temp

print(four[:2, :2, ...])

<a id='eq'></a>
### Присваивание значений срезам
[Вверх](#up)

Помимо изменения значений элементов массивов через обращение по индексам, как в списках, массивы модуля numpy поддерживают удобный механизм присваивания значений элементов через срезы. Т.е. слева указывается желаемый диапазон элементов, которые есть необходимость заменить, а справа (от знака "=") указывается новое значение. 

In [ ]:
import numpy as np

z = np.arange(1, 13)
print('Исходный массив z \n', z)
z[3:6] = 0

print('Изменённый массив z после среза\n', z)

In [ ]:
import numpy as np

one = np.arange(1, 13)
two = np.random.random((10, 20))
four = np.ones((5, 4, 3, 7))

temp = 36.6
two[:, 2] = temp
four[..., 2, 2:5] = temp

print(four[:2, :2, ...])

Присваивать срезу массива можно не только **одно значение**, но любое объект-контейнер, который **по форме (shaoe) будет равен** срезу массива.

In [ ]:
import numpy as np

sh = (2, 3, 5)
z = np.arange(30).reshape(sh)
print('Исходный массив z \n', z)

fslice = z[:, 1, 1:3] 
print(fslice.shape)
new = np.random.randint(-5, 50, size=fslice.size)
rnew = np.reshape(new, fslice.shape)

z[:, 1, 1:3] = rnew

print('Изменённый массив z после среза\n', z)

<a id='copy'></a>
### Копии массивов
[Вверх](#up)

Массивы относятся к изменяемым типам данных - **`mutable data type`**.

Это означает, что без специального копирования (оно может быть поверхностным или глубоким (deep copy)), массив будет сохранять связь со своими "производными", которая будет выражаться в одновременном изменении значений в обоих (или большем) переменных. 

Только глубокие копии позволяют полностью разнести в памяти данные. Сделать глубокую копию массива или его части просто: либо с помощью функции **np.copy(arr)**, либо с помощью метода массива **arr.copy()**.

In [ ]:
import numpy as np

sh = (2, 4)
z = np.arange(8).reshape(sh)
print('Исходный массив z \n', z)

z1 = z   # это просто разные ссылки на одни данные
z2 = z[:]   # поверхностная копия
z3 = z.copy()   # глубокая копия

z[:, 2] = range(-4, -2)
for a in zip(z1, z2, z3):
    print(a)

Обычно все универсальные функции (например, нахождение максимума) возвращают копии, которые не затрагивают исходный массив. Но не стоит забывать про срезы!

In [ ]:
import numpy as np

sh = (2, 4)
z = np.arange(8).reshape(sh)
print('Исходный массив z \n', z)

# Универсальная функция
z1 = z.mean(axis=1)   # это просто разные ссылки на одни данные
z1[0] = 333   # это глубокия копия!

# ВНИМАНИЕ!!! Срез!!! 
z2 = z[0, :]   # срез - это поверхностная копия!
z2[3] = -99   # изменяя срез, изменяет и массив-родитель.

print('Изменяя срез, изменяется и массив-родитель!!!\n', z)


Чтобы избежать "перекрытия" данных, нужно использовать копии.

In [ ]:
import numpy as np

sh = (2, 4)
z = np.arange(8).reshape(sh)
print('Исходный массив z \n', z)

# ВНИМАНИЕ!!! Срез!!! 
z2 = z[0, :].copy()   # копия среза
z2[2:4] = -99   # теперь срез не влияет на массив!

print('Срез изменился, а массив - нет!!!\n', z)

Такая же "история" происходит с `pandas.DataFrame`, когда мы берём срез и получаем `pandas.Series`. Т.к. объект уже другого типа, то часто забывается, что изменения в объекте-Series, приводят к изменениям в родительском объекте-DataFrame. 

<a id='merging'></a>
## Объединение массивов
[Вверх](#up)

Часто требуется объединить значения из нескольких массивов. "Склеивание" или объединение должно проходить по выбранной оси, т.е. объединяем по строкам, столбцам и т.д. 

Чтобы объединить несколько массивов или векторов или массива с векторами, существуют следующие функции:

+ **numpy.concatenate()** - объединяет массивы вдоль существующей оси;
+ **numpy.stack()** - объединяет массивы вдоль **новой** оси;
+ **numpy.vstack()** - объединяет массивы вдоль строк (row wise);
+ **numpy.hstack()** - объединяет массивы вдоль столбцов (column wise);
+ **numpy.dstack()** - объединяет массивы вдоль третьей оси "глубины" (along third dimension).

Наиболее универсальной является **concatenate**, которая принимает сам массив `arr` и номер оси `axis` вдоль которой будет происходить объединение. Объединять вдоль выбранной оси можно только массивы с одинаковой длиной. Так массивы x и y с формами (10, 4) и (7, 4) соответственно можно объединить по нулевой оси (добавить строки массива-y под строки массива-x), т.к. число столбцов совпадает. Но объединить их по первой оси не получится!

Автодополнение (как у объектов pandas.DataFrame) у массивов numpy не поддерживается.

Для объединения по новым осям (т.е. новая ось появляется где-то рядом с уже существующими) используется функция **np.stack**. 
Так для нескольких двумерных массивов можно провести объединение по трём "новым" осям: 0, 1 и 2. Т.е. либо ДО, либо ПОСЛЕ, либо МЕЖДУ существующими осями. В результате число осей исходных массивов увеличивается на один.

> Для объединения по новой оси массивы должны быть одинаковой формы!

In [ ]:
import numpy as np

sh = (3, 4)
x = np.arange(12).reshape(sh)
y = np.ones(sh) - 1.5

print('Исходные формы массивов', x.shape, y.shape)

z0 = np.stack((x, y), axis=0)
z1 = np.stack((x, y), axis=1)
z2 = np.stack((x, y), axis=2)

for i, z in enumerate([z0, z1, z2]):
    print(f'Объединение по новой {i} оси', z.shape)

try:
    z3 = np.stack((x, y), axis=3)
except:
    print('z3 = np.stack((x, y), axis=3) -->\n'
          'AxisError: axis 3 is out of bounds for array of dimension 3!')

В случае когда требуется объединить массивы по существующим осям (т.е. число измерений или осей массива не изменится), объединение также следует проводить вдоль какой-то оси. В случае двумерных массивов объединять можно вдоль строк и вдоль столбцов.

In [ ]:
# Объединение вдоль строк
import numpy as np

sh = (2, 4)
x = np.arange(8).reshape(sh)
y = np.zeros((5, 4))

# Объединяем по строкам! Т.е. один массив сцепляется с предыдущим добавляя свои строки "ниже".
z1 = np.concatenate((x, y), axis=0)
z2 = np.vstack((y, x))
z3 = np.row_stack((x, y))

for z in [z1, z2, z3]:
    print(z.shape)
    print(z)

In [ ]:
# Объединение вдоль столбцов
import numpy as np

sh = (7, 7)
x = np.arange(49).reshape(sh)
y = np.zeros((7, 3))

z1 = np.concatenate((x, y), axis=1)
z2 = np.hstack((y, x))
z3 = np.column_stack((x, y))

for z in [z1, z2, z3]:
    print(z.shape)
    print(z)

<a id='vectors'></a>
### Объединение векторов
[Вверх](#up)

Для объединения **векторов** в массив, а также для объединения вектора и массива существует также ряд функций:

+ **numpy.column_stack()** - объединяет вектора по столбцам;

+ **numpy.row_stack()** - объединяет вектора по строкам;

Зачем они нужны? 

Т.к. вектора - это одномерные массивы, у которых есть только нулевая ось, то объединить их в столбцы по первой оси (которая отвечает за столбцы в функции *concatenate*) не получается. Нужно либо искусственно добавлять новую ось, либо использовать отдельную функцию.

> Добавить новую ось можно на лету с помощью `np.newaxis` прямо внутри квадратных скобок на месте новой оси!

In [ ]:
# Объединяем вектора в массив по столбцам

import numpy as np

sh = (7, 7)

# Три вектора
inx = np.arange(5)
sst = np.zeros(inx.shape)
temp = 273. + 10. * np.random.random(sst.shape)

tup = (inx, sst, temp)
z1 = np.concatenate(tup, axis=0)
z2 = np.hstack(tup)
z3 = np.column_stack(tup)   # только эта функция объединяет вектора по столбцам, а не строкам.

for z in [z1, z2, z3]:
    print(z.shape)
    print(z)

Добавить новую ось можно на лету с помощью **`np.newaxis`** прямо внутри квадратных скобок на местоположении новой оси! 

Альтернативно можно сделать reshape массива и добавить "мнимую" вторую ось длиной 1.

В любом случае вектор (у которых одно ось) станет по форме массивом (с двумя осями). И тогда к нему можно применять выше описанные функции.

In [ ]:
# Объединяем вектора в массив по столбцам "нативными" методами - столбцы
import numpy as np

# Три вектора
inx = np.arange(5)
sst = np.zeros(inx.shape)
temp = 273. + 10. * np.random.random(sst.shape)

print('inx[:] shape is ... {}'.format(inx.shape))
print('inx[:, np.newaxis] shape is ... {}'.format(inx[:, np.newaxis].shape))

temp = temp.reshape(len(temp), 1)   # явный reshape от объекта с одной осью к объекту с двумя осями

tup = (inx[:, np.newaxis], sst[:, np.newaxis], temp)   # на лету меняем форму, добавляя ось

# В случае reshape и np.np.newaxis работают все функции
z1 = np.concatenate(tup, axis=1)
z2 = np.hstack(tup)
z3 = np.column_stack(tup) 

for z in [z1, z2, z3]:
    print(z.shape)
    print(z)

In [ ]:
# Объединяем вектора в массив по столбцам "нативными" методами - строки
import numpy as np

# Три вектора
inx = np.arange(5)
sst = np.zeros(inx.shape)
temp = 273. + 10. * np.random.random(sst.shape)

print('inx[:] shape is ... {}'.format(inx.shape))
print('inx[:, np.newaxis] shape is ... {}'.format(inx[np.newaxis, :].shape))

temp = temp.reshape(1, len(temp))   # явный reshape от объекта с одной осью к объекту с двумя осями

tup = (inx[np.newaxis, :], sst[np.newaxis, :], temp)   # на лету меняем форму, добавляя ось

#tup = (inx, sst)   # на лету меняем форму, добавляя ось

# В случае reshape и np.np.newaxis работают все функции
z1 = np.concatenate(tup, axis=0)
z2 = np.vstack(tup)
z3 = np.row_stack(tup) 

for z in [z1, z2, z3]:
    print(z.shape)
    print(z)

При объединении **вектора с массивом**, нужно пользоваться функциями "как для векторов".

In [ ]:
# Объединяем вектор и массив
import numpy as np

# Три вектора
inx = np.arange(5)
sst = np.zeros((10, 5))

tup = (inx[np.newaxis, :], sst[:])   # на лету меняем форму, добавляя ось

# В случае reshape и np.np.newaxis работают все функции
z1 = np.concatenate(tup, axis=0)
z2 = np.vstack(tup)
z3 = np.row_stack((inx, sst))   # позволяет объединить вектор и массив без явной преобразований формы

for z in [z1, z2, z3]:
    print(z.shape)
    print(z)

<a id='spliting'></a>
## Разделение массивов
[Вверх](#up)

Часто требуется объединить значения из нескольких массивов. "Склеивание" или объединение должно проходить по выбранной оси, т.е. объединяем по строкам, столбцам и т.д. 

Чтобы объединить несколько массивов или векторов или массива с векторами, существуют следующие функции:

+ **numpy.split()** -  разделяет массив на N массивов по заданной оси. Возвращает список массивов;
+ **numpy.vsplit()** - разделяет массив на N массивов вдоль строк (row wise). Возвращает список массивов;
+ **numpy.hsplit()** - разделяет массив на N массивов вдоль столбцов (column wise). Возвращает список массивов;
+ **numpy.dsplit()** - разделяет массив на N массивов вдоль третьей оси "глубины" (along third dimension). Возвращает список массивов.

Наиболее универсальной является **split**, которая принимает сам массив `arr`, число `N` на которое нужно поделить исходный массив вдоль заданой оси и номер этой самой оси `axis`. В качестве N можно передавать не только число, но и границы (через индексы) интервалов индексов, на которые будут разбиты данные вдоль выбранной оси.

Например, разбиение массива arr c формой (5, 10, 3) для оси axis=1, где N определено как N=\[2, 3\] будет осуществлено так:

+ arr[:2]

+ arr[2:3]

+ arr[3:]

Т.е. из одного массива с формой (5, 10, 3), будет три массива (упакованы в список) с формами (5, 2, 3), (5, 1, 3), (5, 7, 3)

In [ ]:
import numpy as np

sh = (5, 10, 3)
arr = np.arange(150).reshape(sh)
print('Форма исходного массива z \n', z.shape)

# new - это список массивов!
new = np.split(arr, [2, 3], axis=1)

for i, z1 in enumerate(new):
    print(f'Новый массив z-{i} \n', z1.shape)

В отличие от векторов, разделить N-мерные массивы можно только на равные части, т.е. тут работает то же правило, что и при reshape.

In [ ]:
# Разделение N-мерных массивов
import numpy as np

sh = (3, 4, 5)
z = np.arange(60).reshape(sh)
print(z.shape)
print('Исходный массив z \n', z)

# new - это список массивов!
new = np.split(z, 2, axis=1)
for i, z1 in enumerate(new):
    print(f'Новый массив z-{i} \n', z1.shape, z1)
    
new2 = np.split(z, 3, axis=0)
for i, z1 in enumerate(new2):
    print(f'Новый массив z-{i} \n', z1.shape, z1)

In [ ]:
# Разделение N-мерных массивов с помощью numpy.hsplit и numpy.vsplit
import numpy as np

sh = (10, 7)
z = np.arange(70).reshape(sh)
print(z.shape)
print('Исходный массив z \n', z)

# new - это список массивов!
new = np.vsplit(z, 2)
for i, z1 in enumerate(new):
    print(f'Новый массив z-{i} \n', z1.shape)
    
new2 = np.hsplit(z, [3])   #  [:3] & [3:]
for i, z1 in enumerate(new2):
    print(f'Новый массив z-{i} \n', z1.shape)

<a id='numpy_functions'></a>
## Универсальные функции модуля numpy
[Вверх](#up)

В модуле `numpy` есть огромное количество функций, которые облегчают работу с массивами и элементами внутри них. Вот только некоторые из них:

+ **np.size(arr)** - число элементов в массиве;

+ **np.max()/np.min()/np.mean()** - простые статистики: максимум, минимум, среднее;

+ **np.std()/np.var()** - стандартное отклонение (сигма)/дисперсия;

+ **np.argmin()/np.argmax()** - возвращают индексы экстремумов массива;

+ **np.sort()** - сортировка массива;

+ **np.argsort()** - возвращает исходные индексы отсортированных элементов;

+ **np.gradient()** - возвращает кортеж градиента по осям массива; 

+ **np.in1d(arr1, arr2)** - сравнивает два массива и возвращает логический массив размера arr1, где True, если есть совпадение, и False - если элементы отличаются.

+ **np.meshgrid()** - позволяет преобразовать каждый из данных векторов в массив с размерностью, где каждая ось отражает переданный вектор. Удобно для рисования географических данных и для создания из пары векторов двух массивов. 

+ **np.all()** - проверяет на истинность условие ко всем элементы массива. Если все значения True, возвращает True.

+ **np.any()** - проверяет на истинность условие ко всем элементы массива. Если хотя бы одно значение True, возвращает True.

И многие другие. Их очень много. Например, `np.abs()`. Более подробно о массивах из модуля numpy можно [узнать из документации](https://docs.scipy.org/doc/numpy/reference/arrays.ndarray.html)

<a id='where'></a>
### Поиск индексов элементов массива по логическим условиям
[Вверх](#up)

Среди универсальных функций есть очень удобный инструмент для поиска элементов по логическим условиям через операции с индексами элементов - **np.where()**.

**np.where(условие Y)** возвращает кортеж индексов массива, для элементов которых удовлетворяется условие Y. Длина кортежа равна числу осей массива, для которого составлено условие Y. Т.е. в случае двумерного массива arr функция np.where для условия (arr > 5) вернёт кортеж длины два, где каждый из элементов кортежа будет состоять из массива индексов элементов, для которых истинно условие > 5, для соответствующей оси массива arr. 

Нужно иметь ввиду, что даже если будет возвращено всего одно значение, оно всё равно будет обёрнуто в кортеж! Чтобы его получить явно, нужно обратиться к элементу кортежа через индекс. 


In [ ]:
import numpy as np

x = np.arange(12)

ii = np.where(x >= 11)   # возвращается всего один элемент
print(type(ii), 'len', len(ii))
print('tuple', ii, 'first elem ot array', ii[0])

print(x[ii[0]])

В случае, когда `numpy.where` возвращает несколько индексов, кортеж оборачивает массив. И чтобы "добраться" до элемента, нужно использовать [] дважды!


In [ ]:
import numpy as np

x = np.arange(12)

ii = np.where(x >= 6)   # возвращается несколько элементов
print(type(ii), 'len', len(ii))

# Дважды обращаемся по нулевым индексам
print('tuple', ii, 'first elem ot array', ii[0])

print(x[ii[0]])

Поэтому в случае, когда возвращаются одномерные массивы, рекомендуется ставить в конце np.where обращение к нулевому элементу:

**`ii = np.where(condition)[0]`**

In [ ]:
import numpy as np

x = np.arange(12)

ii = np.where(x >= 6)[0]  # x - одномерный массив
print(type(ii), 'len', len(ii))
print('tuple', ii, 'first elem ot array', ii[0])

print(x[ii[0]])

**N.B.** Для составления логических цепочек в **np.where()** необходимо использовать символ **&** вместо **and** и символ **|** вместо **or**. Например, так:

> **(x > 10) & (x < 20)** вместо **(x > 10) and (x < 20)**

<a id='vec_slice'></a>
### Векторные срезы по нескольким осям на примере numpy.where
[Вверх](#up)

В случае взятия векторных срезов в многомерном массиве можно было бы ожидать, что как и в одномерном случае, для многомерного векторного среза нужно лишь передать последовательности индексов в квадратные скобки в соответствующее место. 

Увы, это не так. 

Разберём несколько примеров векторных срезов от многомерных массивов с помощью функции **numpy.where**, которая возвращает очень подходящие объекты для взятия срезов.

Срезы индексов для каждой оси можно передавать последовательно, а можно просто использовать логическое условие, из которых получены срезы индексов (np.where).

In [ ]:
import numpy as np

sh = (4, 6, 9)
arr = np.random.random(36*6).reshape(sh)*10.

cond = (arr >= 5)

x, y, z = np.where(cond)   # передаём массивы индексов для каждой оси массива
a = np.where(cond)   # a - кортеж
box = [x, y, z]
for i, b in enumerate(box):
    print(len(b), type(b), b[:7])
    
brr = arr[cond]   # подставляем условие
crr = arr[x, y, z]   # подставляем срезы в каждую ось
drr = arr[a]

box2 = [brr, crr, drr]
for j, b2 in enumerate(box2):
    print('j', j, b2.shape)

print('Результаты совпадают :', np.sum(brr - crr), np.sum(brr - drr))  # результат одинаковый

Обычные срезы берутся также, как и у векторов: в каждую ось задаётся свой интервал или индекс.

In [ ]:
# Пример взятия обычных срезов от многомерного массива
import numpy as np

sh = (4, 6, 9)
arr = np.random.random(36*6).reshape(sh)*10.

slices = arr[0:2, 2:, :5]
print(slices.shape)

Прихотливые срезы (fancy slices, они же векторные срезы) позволяют получить массивы формы, отличной от формы исходного массива.

Это отлично работает для векторов, одномерных массивов.

In [ ]:
# Fancy slicing

import numpy as np

arr = np.arange(3)
a = [0, 0, 0, 1]
b = arr[a]

print('Before:', arr, arr.shape)
print('After "fancy" slicing', b, b.shape)

В случае многомерных массивов, прихотливые срезы можно применять лишь последовательно. Одновременно передать несколько прихотливых срезов по разным осям не удаётся. 

Можно скомбинировать обычный срез, прихотливый срез и все значения (комбинацией двоеточий или с помощью символа троеточия, называемого `ellipsis`).

In [ ]:
# Комбинирование обычного среза, прихотливого среза и индекса

import numpy as np

sh = (4, 6, 9)
arr = np.random.random(36*6).reshape(sh)*10.
print('Original shape', arr.shape)

x = [0, 2]
y = [2, 3, 4, 0]
z = [1, 0, 2, 4, 7, 1]

s2 = arr[x, 3:5, :]
s3 = arr[..., z]
s4 = arr[:, y, 8]

for i, s in enumerate([s2, s3, s4]):
    print(f's{i+2} shape', s.shape)

Но одновременно применять к разным осям комбинации индексов в виде списков или массивов не удаётся!

In [ ]:
import numpy as np

sh = (4, 6, 9)
arr = np.random.random(36*6).reshape(sh)*10.
print('Original shape', arr.shape)

x = (0, 2)
y = (0, 2, 3, 4)
z = [0, 2, 3, 4, 7]

try:
    s2 = arr[x, y, :]
except:
    print('ERROR! Одновременно применять два fancy slice запрещается!')

try:
    s3 = arr[0, y, z]
except:
    print('ERROR! Одновременно применять два fancy slice запрещается!')

try:
    s4 = arr[0, 0:3, z]
    print('Можно использовать лишь fancy slice в квадратных скобках!')
except:
    print('ERROR! Одновременно применять два fancy slice запрещается!')

    #s3 = s2[:, y, :]
#s4 = s3[..., z]

#for i, s in enumerate([s2, s3, s4]):
#    print(f's{i+2} shape', s.shape)

В отличие от обычных срезов, **`многомерные векторные срезы`** нужно брать ПОСЛЕДОВАТЕЛЬНО!

In [ ]:
# Взятие векторных срезов от многомерных массивов

import numpy as np

sh = (4, 6, 9)
arr = np.random.random(36*6).reshape(sh)*10.

x = [0, 2, 1]*3
y = [2, 3, 4, 0]
z = [1, 0, 2, 4, 7, 1]

# ПОСЛЕДОВАТЕЛЬНО!
s2 = arr[x, ...]   # раз 
s3 = s2[:, y, :]   # два
s4 = s3[..., z]   # три

print(s4.shape, s4[:3,:3,:3])

<a id='logic_fancy'></a>
### Логические условия и fancy-slicing
[Вверх](#up)

Логические условия, как и fancy-slicing, можно применять лишь последовательно, но не одновременно в одних квадратных скобках.

In [ ]:
import numpy as np

sh = (4, 6, 9)
arr = np.random.random(36*6).reshape(sh)*10.
print('Original shape', arr.shape)

condX = arr[:, 0, 0] > 3   # логическое условие для нулевой оси
print(condX)
y = [0, 2, 3, 4]

try:
    s2 = arr[condX, y, :]
except:
    s2 = arr[condX, ...][:, y, :]

print('s2 shape', s2.shape)
    #s3 = s2[:, y, :]
#s4 = s3[..., z]

#for i, s in enumerate([s2, s3, s4]):
#    print(f's{i+2} shape', s.shape)

<a id='geo'></a>
### Пример с геоданными
[Вверх](#up)

Рассмотрим пример для данных в регулярной географической сетке с регулярным шагом 2.5 градуса. Необходимо выбрать данные в заданном регионе [lat1 : lat2, lon1 : lon2].

In [ ]:
import numpy as np

lon = np.arange(-180, 180, 2.5)
lat = np.arange(-90, 90 + 0.1, 2.5)

zarr = np.random.random((lat.shape[0], lon.shape[0]))

lat1 = 40.
lat2 = 85.
lon1 = 0.
lon2 = 90.

jj = np.where((lat > lat1) & (lat < lat2))
ii = np.where((lon > lon1) & (lon < lon2))

try:
    varr = zarr[jj, ii]
    print(varr.shape)
except:
    print("Error! ii - кортеж с длиной один. Единственный элемент такого кортежа - массив индексов с длиной", len(ii[0]))

i1 = ii[0]   # i1 - массив длиной 35
j1 = jj[0]   # j1 - массив длиной 17

try:
    varr = zarr[j1, i1]
    print(varr.shape)
except:
    print("Error! i1 - массив, который нельзя использовать одновременно с массивом индексов j1!")

# Последовательно берём срезы по осям
try:
    z1 = zarr[j1, :] 
    varr = z1[:, i1]
    print(varr.shape)
    print('Только последовательность срезов даёт адекватный результат')
    print(('Есть возможность "сцеплять" как у многомерных списков последовательность взятия срезов.'
           'Но это более запутаный с точки зрения синтаксиса по мнению автора способ взятия среза.'))
    varr2 = zarr[j1, :][:, i1]
    print(varr2.shape)
    
    
except:
    print("Тут ошибки не будет!")



Однако, одновременно в одних квадратных скобках можно взять два обычных среза!

In [ ]:
import numpy as np

lon = np.arange(-180, 180, 2.5)
lat = np.arange(-90, 90 + 0.1, 2.5)

zarr = np.random.random((lat.shape[0], lon.shape[0]))

jj = np.where((lat > 40) & (lat < 85))
ii = np.where((lon > 0.) & (lon < 90.))

i1 = ii[0][0]
i2 = ii[0][-1]

j1 = jj[0][0]
j2 = jj[0][-1]

varr = zarr[j1 : j2 + 1, i1 : i2 + 1]
print(varr.shape)

Также не работают логические маски-условия при одновременном использовании в одних квадратных скобках срезов.

In [ ]:
import numpy as np

lon = np.arange(-180, 180, 2.5)
lat = np.arange(-90, 90 + 0.1, 2.5)

zarr = np.random.random((lat.shape[0], lon.shape[0]))

condJ = (lat > 40) & (lat < 85)
condI = (lon > 0.) & (lon < 90.)

try:
    varr = zarr[condJ, condI]    
except:
    print('Рабоает только последовательное наложение логических условий на оси:')
    z1 = zarr[condJ, :]
    varr = z1[:, condI]
    print(varr.shape)
    
    varr2 = zarr[condJ, ...][:, condI]
    print(varr2.shape)

<a id='masked'></a>
## Массивы с логической маской
[Вверх](#up)

<a id='ma'></a>
### Пространство numpy.ma
[Вверх](#up)

Для применения математических функций чаще всего требуются "полные данные", т.е. данные, в которых нет пропусков, все значения лежат в области изменения функции, являются однотипными и т.д.

Такая полнота данных суть идеализированный случай. Реальные наборы данных часто содержат:

+ пропуски;

+ ошибочные значения;

+ значения, очень сильно отличающиеся от соседних значений;

+ некорректные значения.

Для восстановления полноты данных в модуле numpy существует `пространство ma`, которое позволяет применять почти все универсальные функции numpy к данным-массивам, в которых встречаются выше перечисленные проблемы.





In [ ]:
import numpy as np
from numpy import ma

marr = ma.arange(12)
marr2 = np.ma.arange(12)

print(type(marr), type(marr2))

Пространство ma вводит разновидность numpy-массивов - **MaskedArray** или "массив с маской", "масочный массив". Этот тип данных очень похож на обычный массив, у которого всегда дополнительно есть ещё и логический массив (массив из значений True/False). По умолчанию все элементы такого логического массива-маски (или просто маски) имеют значения **False**.

При создании масочного массива или позже можно указать в качестве аргумента/атрибута *mask* логическое условие. Это условие будет применено к каждому элементу исходного массива и, если значение элемента будет удовлетворять условию, то для такого элемента значение маски массива будет True. Те элементы массива, которые не удовлетворяют условию, получат значения маски False. Т.о. будет сформирована логическая маска массива.

In [ ]:
import numpy as np

x = np.arange(12)   # обычный массив
print(x)
marr = np.ma.array(x, mask=(x % 2 == 0))   # масочный массив

print(type(marr), marr)
marr[0] = marr[0] + 1
print(marr.data)

marr[marr.mask] = range(-6, 0)
print(marr)

На основе логической маски массива, все True элементы "маскируются" и больше не участвуют в арифметико-логических операциях масочного массива. При печати на экран они обозначаются двумя тире "--". Это не значит, что данные стёрты! Нет, их можно получить с помощью атрибута масочного массива **data (arr.data)**. Просто они скрыты, замаскированы для удобства расчётов.

In [ ]:
import numpy as np
from numpy import ma

marr = ma.arange(12)

# Создаём логическую маску ()
marr.mask = marr < 4

# Массив-маска
print('marr     ', marr)
# Маска массива
print('marr.mask', marr.mask)
# Значения элементов массива
print('marr.data', marr.data)

Рассмотрим случай плохо структурированных данных (взятых из файла, например), где значения некоторых столбцов в строках отсутствуют. Тем не менее, необходимо провести расчёты каких-либо статистик на основе даже неполной информации. Этот вариант описывает пропуски в данных. Пропуски можно проигнорировать и провести расчёты по оставшимся значениям, а можно как-то их заполнить, заменить на какие-то значения. Но восполнение целостности данных с помощью методов интерполяции или заполнения пропусков другими значениями - это тема, которую мы частично рассмотрим ниже. Здесь же исследуем только случаи игнорирования пропусков.

Наложение маски на пропущенные значения - один из вариантов проведения расчётов по данным с пропусками. Пропуски же возникают чаще всего при чтении плохо структурированных данных и при возникновении ошибок вычислений. 

<a id='nan'></a>
### numpy.NaN
[Вверх](#up)

В numpy есть особый объект - `np.nan`, что означает "Это не число!" ("Not A Number"). Для чего он нужен? Для бесшовной работы с числовыми типами данных (т.е. без необходимости перехода к другим типам данных, например, `None`), значений которых нет или их не следует рассматривать при арифметико-логических операциях. [Более подробно с историй возникновения и введения в практику nan-ов - тут](https://en.wikipedia.org/wiki/NaN).

Возникают nan-ы при чтении данных (заполняют отсутствующие данные - см. пример с файлом *"w7_nums.txt"*), возникают как результат вычислений, вышедших из русла разумного и возможного (например, при обращении матрицы). Когда `nan-ы` есть в массиве, то вычисления с ним становятся мукой. Любая операция с nan возвращает также nan. Т.е. достаточно появиться хотя бы одному np.nan в цепочке вычислений, чтобы итогом стало также **nan**.

Nan-ы необходимо маскировать или экранировать для устойчивых вычислений. Безопасно проверить элемент или переменную на принадлежность к nan можно с помощью функции **np.isnan()**.

Используя функцию `np.isnan()` в выражении маски для масочного массива можно экранировать nan-ы и продолжить вычисления.

In [ ]:
# Пример с np.nan
import numpy as np

arr = np.array([1., np.nan, 5.3, -0.9, np.nan, -0.3 ])   # обычный массив
print('Is arr[1] == np.nan?', np.isnan(arr[1]))
print(type(arr), arr)

print('Mean arr:', np.mean(arr))

# Маскируем nans
marr = np.ma.array(arr, mask=np.isnan(arr))   # масочный массив
print(type(marr), marr)
print('Mean marr:', np.mean(marr))

In [ ]:
# Чтение плохо структурированного файла w7_nums.txt
import numpy as np

arr = np.genfromtxt('./data/w7_nums.txt', delimiter=';')
print('Оригинал', arr.shape)
print(arr)

marr = np.ma.array(arr, mask= (np.isnan(arr)) | (arr < -900))
print('После наложения маски', marr.shape)
print(marr)

<a id='filled'></a>
### Заполнение замаскированных значений
[Вверх](#up)

Помимо игнорирования пропусков в данных, можно попробовать заменить пропущенные значения на какие-то значения. В случае использования масочных массивов задача сводится к заполнению замаскированных значений. 

Для восстановления целостности данных в массивах-масках, соответствующие True-значениям в маске элементы, можно применить несколько методов массивов-масок:

+ **marr.fill_value** - определяет значение, которым будут заполнены замаскированные значения;

+ **marr.filled()** - заполняет замаскированные значения с помощью атрибута fill_value или переданного значения.


In [ ]:
import numpy as np

arr = np.array([1., np.nan, 5.3, -0.9, np.nan, -0.3 ])

marr = np.ma.array(arr, mask=((arr >= 5) | np.isnan(arr)))   # масочный массив

print('Оригинал', marr)
a2 = marr.filled(marr.mean())   # заполнение средним значением (по трём valid значениям)
print('Заполнение константным методом', a2)

In [ ]:
import numpy as np

arr = np.arange(12)   # обычный массив
marr = np.ma.array(arr, mask=(x <= 3), fill_value=-99)   # масочный массив

print('Оригинал', marr)

a1 = marr.filled()
print('Заполнение с помощью атрибута fill_value', a1)

marr.fill_value = 77
a2 = marr.filled()
print('Заполнение с помощью атрибута fill_value', a2)

a3 = marr.filled(0)
print('Заполнение константным методом', a3)

Помимо заполнения или игнорирования пропусков, полнота данных также зависит от препроцессинга, подготовки данных, проверки на "вылеты" и экстремальные значения (которые могут не иметь физического смысла, а являться ошибкой при записи, копировании, передаче данных). Это перетекает в задачу наложения логического условия на значения элементов массива. 

Т.е. после заполнения пропусков механизм логической маски можно использовать для маскирования т.н. invalid данных (например, значений над континентами для массивов потоков тепла океана). И не заполнять их, если это не несёт физического смысла.

В случае же анализа на неадекватные или ошибочные значения, можно повторить процедуру заполнения данных разумными значениями.

<a id='print_numpy'></a>
## Управляемый вывод представления чисел
[Вверх](#up)

[Печать массивов с сайта PythonWorld](https://pythonworld.ru/numpy/1.html)

Numpy много работает с действительными (и не только) числами. Часто на печать выводится много лишней числовой информации (вроде седьмого знака после запятой).

Модуль **numpy** позволяет управлять выводом на экран числовой информации с помощью спецальной функции:

> **np.set_printoptions()**

Функция принимает ряд аргументов, среди которых: 

+ **precision** - количество отображаемых цифр после запятой (по умолчанию 8);

+ **threshold** - количество элементов в массиве, вызывающее разрыв массива при выводе на экран (по умолчанию 1000);

+ **edgeitems** - количество элементов в начале и в конце каждой размерности массива (по умолчанию 3);

+ **linewidth** - количество символов в строке, после которых осуществляется перенос (по умолчанию 75);

+ **suppress** - если True, то не выводит на экран небольшие значения в scientific notation (по умолчанию False);

+ **nanstr** - строковое представление NaN (по умолчанию 'nan');

+ **infstr** - строковое представление Inf (по умолчанию 'inf');

+ **sign** - позволяет выводить унарные знаки (плюс, минус и пробел) перед числами (по умолчанию '-')

In [ ]:
import numpy as np

z = np.random.random((40, 20))*10.
print('До "урезания" знаков после запятой до 3\n', z)

np.set_printoptions(precision=3, sign='+')
print('После (3 знака после запятой), все положительные со знаком +\n', z)

[Наверх](#up)